In [ ]:
# !pip install git+https://github.com/huggingface/diffusers transformers accelerate datasets xformers
# !pip install tomesd

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-26sddt5l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-26sddt5l
  Resolved https://github.com/huggingface/diffusers to commit bc7a4d4917456afd70913be85bd25c556c25862c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# !pip install git+https://github.com/Navu45/image-merging.git

  Cloning https://github.com/Navu45/image-merging.git to /tmp/pip-req-build-k2wlbmfu
  Running command git clone --filter=blob:none --quiet https://github.com/Navu45/image-merging.git /tmp/pip-req-build-k2wlbmfu
  Resolved https://github.com/Navu45/image-merging.git to commit c0fd3a04df149dfab503dda49ea8d35ba041540e
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import random

import torch
from PIL import Image
from diffusers import KandinskyV22PriorEmb2EmbPipeline, DPMSolverMultistepScheduler
from diffusers.models import UNet2DConditionModel, VQModel
from diffusers.schedulers import DDPMScheduler
from diffusers.utils import make_image_grid
from transformers import (
    CLIPSegProcessor, CLIPSegForImageSegmentation, AutoImageProcessor,
    MobileViTV2ForImageClassification, CLIPImageProcessor, CLIPVisionModelWithProjection,
)
import tomesd
from models.combined_pipeline_prior import CombinedPipelineV2


    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [ ]:
# segmentation_processor = CLIPSegProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
# segmentation_model = CLIPSegForImageSegmentation.from_pretrained("CIDAS/clipseg-rd64-refined")
# classifier_processor = AutoImageProcessor.from_pretrained("apple/mobilevitv2-1.0-imagenet1k-256")
# image_encoder = CLIPVisionModelWithProjection.from_pretrained('kandinsky-community/kandinsky-2-2-prior',
#                                                               subfolder="image_encoder", use_safetensors=True)
# image_processor = CLIPImageProcessor.from_pretrained('kandinsky-community/kandinsky-2-2-prior',
#                                                      subfolder="image_processor")
# classifier_model = MobileViTV2ForImageClassification.from_pretrained("apple/mobilevitv2-1.0-imagenet1k-256")

# movq = VQModel.from_pretrained('kandinsky-community/kandinsky-2-2-decoder-inpaint',
#                                 subfolder="movq", use_safetensors=True)
# unet = UNet2DConditionModel.from_pretrained('kandinsky-community/kandinsky-2-2-decoder-inpaint',
#                                             subfolder="unet", use_safetensors=True)

# noise_scheduler = DDPMScheduler.from_pretrained('kandinsky-community/kandinsky-2-2-decoder-inpaint', subfolder="scheduler")

# decoder = CombinedPipelineV2(
#     unet,
#     noise_scheduler,
#     movq,
#     image_encoder=image_encoder,
#     image_processor=image_processor,
#     segmentation_processor=segmentation_processor,
#     segmentation_model=segmentation_model,
#     classifier_processor=classifier_processor,
#     classifier_model=classifier_model
# )

In [ ]:
# decoder.save_pretrained(save_directory='model/')

In [ ]:
decoder = CombinedPipelineV2.from_pretrained("model/", torch_dtype=torch.float16, variant="fp16", use_safetensors=True)
decoder.scheduler = DPMSolverMultistepScheduler.from_config(decoder.scheduler.config)

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
decoder.enable_model_cpu_offload()
decoder.enable_attention_slicing()

In [ ]:
prior = KandinskyV22PriorEmb2EmbPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-prior',
                                                         image_encoder=decoder.image_encoder,
                                                         image_processor=decoder.image_processor,
                                                         torch_dtype=torch.float16)
prior.enable_model_cpu_offload()
prior.enable_attention_slicing()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
negative_prior_prompt ='lowres, text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature'

clothes = ['shirt', 'pants', 'shorts', 'shoes', 't-shirt',
            'sweatshirt', "cowboy boot", "cowboy hat",
            'crash helmet', 'cardigan', 'blouse', 'jacket', 'jeans', 'top']
torch.manual_seed(42)

In [ ]:
torch.no_grad()
def generate_with_prior(source_image, target_image):
    source_clip_img_emb = prior.interpolate(images_and_prompts=[source_image, target_image],
                                     weights=[1, .8],
                                     num_images_per_prompt=1).image_embeds
    source_image_embeds = prior(prompt='a human being wearing some clothes in front of a white background',
                    image=source_clip_img_emb,
                    strength=0.85,
                    num_inference_steps=25,
                    num_images_per_prompt=1,)
    source_negative_embeds = prior(prompt=negative_prior_prompt,
                         image=source_clip_img_emb,
                         strength=1,
                         num_inference_steps=25,
                         num_images_per_prompt=1)
    target_clip_img_emb = prior.interpolate(images_and_prompts=[target_image], weights=[1], num_images_per_prompt=1, ).image_embeds
    target_image_embeds = prior(prompt='a clothing worn in front of a white background',
                image=target_clip_img_emb,
                strength=0.85,
                num_inference_steps=25,
                num_images_per_prompt=1,)
    target_negative_embeds = prior(prompt=negative_prior_prompt,
                         image=target_clip_img_emb,
                         strength=1,
                         num_inference_steps=25,
                         num_images_per_prompt=1)

    images = decoder(source_image,
                     target_image,
                     source_image_embeds=source_image_embeds.image_embeds,
                     source_negative_image_embeds=source_negative_embeds.image_embeds,
                     target_image_embeds=target_image_embeds.image_embeds,
                     target_negative_image_embeds=target_negative_embeds.image_embeds,
                     source_prompt=clothes,
                     source_negative_prompt=['head, neck', 'hair', 'face', 'hand'],
                     target_prompt=clothes,
                     target_negative_prompt=['head, neck', 'hair', 'face', 'hand',],
                     num_inference_steps=50, strength=0.5,height=768, width=768)
    return images


In [ ]:
input_folder = './'
output_folder = 'out/'

In [ ]:
paths = os.listdir(input_folder)
random.shuffle(paths)

images = [Image.open(input_folder + path) for path in paths if path.endswith('.png')]
generated_images = []

for i in range(len(images)):
    for j in range(i + 1, len(images)):
        images[i], images[j] = images[i].resize((512, 512)), images[j].resize((512, 512))
        generated_images.extend([
            make_image_grid([*generate_with_prior(images[i], images[j]),
                             images[i], images[j]], 2, 2),
            make_image_grid([*generate_with_prior(images[j], images[i]),
                             images[j], images[i]], 2, 2)
        ])

  0%|          | 0/21 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature']


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature']


...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature']


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
def save_images(images, path=''):
    for i, image in enumerate(images):
        image.save(path + f'image{i}.png')

save_images(generated_images, path=output_folder)